
# C2 — Web Scraping (Tottus · Arroz) — Versión 2

**Objetivo:** Extraer productos de la categoría **Arroz** en Tottus, guardar un **CSV** y mostrar una **EDA mínima**.  
Incluye **manejador de errores** para evitar `EmptyDataError` cuando no se extraen productos.


In [1]:

# === Celda 1: Instalación (opcional) e Importaciones ===
# Descomenta si necesitas instalar en tu entorno (Colab/entorno limpio):
!pip install -q requests beautifulsoup4 lxml pandas

import os, re, time, random, json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs, urlencode, urlunparse

BASE_URL = "https://www.tottus.com.pe"
START_URL = "https://www.tottus.com.pe/tottus-pe/lista/CATG16815/Arroz"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
    "Accept-Language": "es-PE,es;q=0.9,en;q=0.8",
}

OUT_DIR = "data"
OUT_CSV = os.path.join(OUT_DIR, "tottus_arroz.csv")
OUT_JSON = os.path.join(OUT_DIR, "tottus_arroz.json")

os.makedirs(OUT_DIR, exist_ok=True)

def clean_text(s):
    if s is None: 
        return None
    s = re.sub(r"\s+", " ", str(s)).strip()
    return s or None

def normalize_price(s):
    if s is None:
        return None
    m = re.findall(r"[\d\.,]+", str(s))
    if not m: 
        return None
    val = m[-1]
    # Heurística para 1.234,56 o 1,234.56 o 1234
    if val.count(",") and val.count("."):
        val = val.replace(".", "").replace(",", ".")
    else:
        val = val.replace(",", "")
    try:
        return float(val)
    except:
        return None



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:

# === Celda 2: Web Scraping + Export CSV/JSON (con manejo de errores) ===

def get_page(url, max_tries=3, sleep_base=1.2):
    """Descarga una página con pequeños reintentos y retorna el HTML en texto."""
    for i in range(max_tries):
        try:
            r = requests.get(url, headers=HEADERS, timeout=25)
            r.raise_for_status()
            return r.text
        except requests.RequestException as e:
            if i == max_tries - 1:
                raise
            time.sleep(sleep_base + i)

def select_first_text(node, selectors):
    """Prueba una lista de selectores CSS y retorna el texto del primero que exista."""
    for sel in selectors:
        el = node.select_one(sel)
        if el:
            txt = el.get("title") or el.get_text(strip=True)
            txt = clean_text(txt)
            if txt:
                return txt
    return None

def select_first_href(node, selectors):
    for sel in selectors:
        el = node.select_one(sel)
        if el and el.get("href"):
            return el.get("href")
    return None

def parse_list(html, base=BASE_URL):
    """Extrae productos desde HTML de la categoría (estrategia defensiva + fallback JSON-LD)."""
    soup = BeautifulSoup(html, "lxml")
    items = []

    # 1) Tarjetas de producto (clases comunes VTEX/Tottus)
    cards = soup.select('[data-component="productSummary"], .product-card, .vtex-product-summary-2-x-container')
    for c in cards:
        title = select_first_text(
            c, ['a[title]', '.product-title', '.vtex-product-summary-2-x-productBrand', 'h3 a']
        )
        price = select_first_text(
            c, ['.price', '.sellingPrice', '.vtex-store-components-3-x-sellingPrice', 'span[class*="sellingPrice"]', '[class*="price"]']
        )
        href  = select_first_href(c, ['a[href]'])
        url   = urljoin(base, href) if href else None
        brand = select_first_text(c, ['.brand', '.vtex-product-summary-2-x-brandName', '[data-brand]'])
        cat_el   = soup.select_one('nav.breadcrumb, .breadcrumb, [aria-label="breadcrumb"]')
        categ = clean_text(cat_el.get_text(" > ", strip=True) if cat_el else "Arroz")

        items.append({
            "titulo": title,
            "precio": price,
            "precio_num": normalize_price(price),
            "detalle_url": url,
            "marca": brand,
            "categoria": categ,
            "tienda": "Tottus",
        })

    # 2) Fallback: JSON-LD (datos estructurados) si no encontró tarjetas
    if not items:
        for script in soup.select('script[type="application/ld+json"]'):
            try:
                data = json.loads(script.string or "")
            except Exception:
                continue

            def as_list(x):
                if isinstance(x, list): return x
                elif x is not None: return [x]
                return []

            for p in as_list(data if isinstance(data, list) else [data]):
                if isinstance(p, dict) and p.get("@type") in ("Product", "ItemList"):
                    if p.get("@type") == "ItemList" and "itemListElement" in p:
                        for it in as_list(p["itemListElement"]):
                            prod = it.get("item") if isinstance(it, dict) else None
                            if not isinstance(prod, dict): 
                                continue
                            name = clean_text(prod.get("name"))
                            url = prod.get("url")
                            offer = prod.get("offers") or {}
                            price = clean_text(str(offer.get("price"))) if isinstance(offer, dict) else None
                            items.append({
                                "titulo": name,
                                "precio": price,
                                "precio_num": normalize_price(price),
                                "detalle_url": url,
                                "marca": clean_text(prod.get("brand", {}).get("name") if isinstance(prod.get("brand"), dict) else prod.get("brand")),
                                "categoria": "Arroz",
                                "tienda": "Tottus",
                            })
                    elif p.get("@type") == "Product":
                        name = clean_text(p.get("name"))
                        url = p.get("url")
                        offer = p.get("offers") or {}
                        price = clean_text(str(offer.get("price"))) if isinstance(offer, dict) else None
                        items.append({
                            "titulo": name,
                            "precio": price,
                            "precio_num": normalize_price(price),
                            "detalle_url": url,
                            "marca": clean_text(p.get("brand", {}).get("name") if isinstance(p.get("brand"), dict) else p.get("brand")),
                            "categoria": "Arroz",
                            "tienda": "Tottus",
                        })

    # Limpieza final (duplicados, nulos)
    clean_items = []
    seen = set()
    for it in items:
        key = (it.get("titulo"), it.get("detalle_url"))
        if key not in seen and (it.get("titulo") or it.get("detalle_url")):
            seen.add(key)
            clean_items.append(it)
    return clean_items

def next_page_url(url, page_num):
    """Construye ?page=N manteniendo otros parámetros."""
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    qs["page"] = [str(page_num)]
    new_query = urlencode(qs, doseq=True)
    return urlunparse((parsed.scheme, parsed.netloc, parsed.path, parsed.params, new_query, parsed.fragment))

# === Ejecutar scraping con paginación simple ===
MAX_PAGES = 8  # Ajusta si quieres más páginas
all_rows = []
for page in range(1, MAX_PAGES + 1):
    url = START_URL if page == 1 else next_page_url(START_URL, page)
    print(f"[{page}] GET {url}")
    html = get_page(url)
    batch = parse_list(html, base=BASE_URL)
    print(f"   ↳ {len(batch)} productos")
    if page > 1 and not batch:
        print("   (No se encontraron más productos, deteniendo paginación).")
        break
    all_rows.extend(batch)
    time.sleep(random.uniform(1.1, 2.0))  # cortesía

# Convertir a DataFrame y guardar si hay datos
df = pd.DataFrame(all_rows).drop_duplicates(subset=["titulo", "detalle_url"])

if df.empty:
    print("\n⚠️ No se extrajeron productos. No se generará CSV vacío para evitar errores posteriores.")
else:
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    with open(OUT_JSON, "w", encoding="utf-8") as f:
        json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    print(f"\n✅ Productos únicos: {len(df)}")
    print(f"💾 CSV:  {OUT_CSV}")
    print(f"💾 JSON: {OUT_JSON}")


[1] GET https://www.tottus.com.pe/tottus-pe/lista/CATG16815/Arroz
   ↳ 0 productos
[2] GET https://www.tottus.com.pe/tottus-pe/lista/CATG16815/Arroz?page=2
   ↳ 0 productos
   (No se encontraron más productos, deteniendo paginación).

⚠️ No se extrajeron productos. No se generará CSV vacío para evitar errores posteriores.


In [3]:

# === Celda 3: Vista previa y EDA mínima (segura) ===

import os
import pandas as pd
import matplotlib.pyplot as plt

csv_path = "data/tottus_arroz.csv"

if not os.path.exists(csv_path) or os.path.getsize(csv_path) == 0:
    print("⚠️ No hay CSV para leer (posiblemente no se extrajeron productos).")
else:
    df = pd.read_csv(csv_path)

    # Vista rápida
    display(df.head(10))
    print("\nFilas:", len(df))
    print("Columnas:", list(df.columns))

    # Resumen de precios
    if "precio_num" in df.columns and df["precio_num"].notna().any():
        print("\nPrecio — resumen:")
        display(df["precio_num"].describe())

    # Top marcas (si existen)
    if "marca" in df.columns:
        top_marcas = df["marca"].fillna("Sin marca").value_counts().head(10)
        if not top_marcas.empty:
            print("\nTop 10 marcas por # de productos:")
            display(top_marcas)

            # Gráfico simple
            plt.figure()
            top_marcas.sort_values(ascending=True).plot(kind="barh")
            plt.title("Top 10 marcas (conteo de productos)")
            plt.xlabel("Productos")
            plt.ylabel("Marca")
            plt.tight_layout()
            plt.show()


⚠️ No hay CSV para leer (posiblemente no se extrajeron productos).
